In [70]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [71]:
dict_table = pickle.load(open('DictGroups2022', 'rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

# 1 Calcular Team Strength

Representamos cada equipo por su "fuerza", es decir, determinamos esta característica de acuerdo a los goles que anotó y los que recibió.

In [72]:
# dividir df en df_home y df_away
df_home = df_historical_data[['HomeTeam', 'HomeScore', 'AwayScore']]
df_away = df_historical_data[['AwayTeam', 'HomeScore', 'AwayScore']]

In [73]:
# renombrar columnas
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeScore': 'Scored', 'AwayScore': 'Conceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeScore': 'Conceded', 'AwayScore': 'Scored'})

In [74]:
# concatenar df_home y df_away, hacer group por team y calcular promedio
# Representamos el "strength" de  cada equipo con el promedio de sus goles anotas y recibidos
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_team_strength

,Scored,Conceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.727273,1.181818
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.634615,1.326923
Wales,0.800000,0.800000
West Germany,2.071429,1.267857


# 2 Función predict_points

La distribución de Poisson es una distribución de probabilidad discreta que expresa, a partir de una frecuencia de ocurrencia media, la probabilidad de que ocurra un determinado número de eventos durante cierto período de tiempo.

**Gol: es un evento que puede ocurrir en los 90 minutos de un partido de fútbol**

Además esta distribución cumple con ciertas condiciones:
1. El número de eventos se puede contar. En un partido existen 1 2 3... goles mientras que no pueden haber 1.4, 2.8... goles.
2. La ocurrencia de eventos es independiente. Es decir, la ocurrencia de un gol no afecta la probabilidad de otro gol.
3. La tasa a la que ocurren los eventos es constante. La probabilidad de un gol es la misma en un partido de 90 minutos que en uno de 20 minutos.
4. Dos eventos no pueden ocurrir exactamente en el mismo instante de tiempo. Es imposible que hayan dos goles en un mismo intervalo de tiempo.

In [75]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index: 
        # fortaleza de local * debilidades de visitante
        # mientras más goles anotas más grande el lambda (fortaleza)
        # mientras más goles recibidos más grande el lambda (debilidad)
        lamb_home = df_team_strength.at[home, 'Scored'] * df_team_strength.at[away, 'Conceded']
        lamb_away = df_team_strength.at[away, 'Scored'] * df_team_strength.at[home, 'Conceded']
        
        
        # el bucle for simula todos los posibles resultados en un rango de 0 a 10
        # es decir: 0-1, 0-2, 0-3...0-10
        # realiza una serie de combinaciones
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): # number of goals home team
            for y in range(0,11): # number of goals away team
                # "p" es la probabilidad
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p # probabilidad se acumula en el empate
                elif x > y:
                    prob_home += p # probabilidad se acumula en el local
                else:
                    prob_away += p # probabilidad se acumula en el visitante
        
        
        # se calcula el puntaje de probabilidad de ganar por cada equipo
        # el puntaje va de 0 - 3 por conveniencia
        # 0=perder, 1=empatar, 3=ganar
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    
    else:
        return (0, 0)

## 2.1 Test de la función

In [76]:
# Test partidos con: Argentina-Mexico, England-United States, Qatar (H)-Ecuador
predict_points('Argentina', 'Ecuador')

(1.8473056096166938, 0.9293177072291673)

# 3 Prediciendo Mundial

## 3.1 Fase de Grupos

In [77]:
# dividiendo el fixture en grupos, octavos, cuartos, ...
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

df_fixture_quarter

,home,away,score,year
56,Winners Match 53,Winners Match 54,Match 58,2022
57,Winners Match 49,Winners Match 50,Match 57,2022
58,Winners Match 55,Winners Match 56,Match 60,2022
59,Winners Match 51,Winners Match 52,Match 59,2022


In [78]:
# for group in dict_table:
#     print(dict_table[group]['Team'].values)

# test de función isin()
df_fixture_group_48[df_fixture_group_48['home'].isin(['Argentina', 'Poland', 'Mexico', 'Saudi Arabia'])]

,home,away,score,year
12,Argentina,Saudi Arabia,Match 8,2022
13,Mexico,Poland,Match 7,2022
14,Poland,Saudi Arabia,Match 22,2022
15,Argentina,Mexico,Match 24,2022
16,Poland,Argentina,Match 39,2022
17,Saudi Arabia,Mexico,Match 40,2022


In [79]:
# recorrer todos los partidos de la fase de grupos y actualizar las tablas de cada grupo
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    # se filtran los 6 partidos por cada grupo con el fin de no simular partidos que no se darán
    # se hacen combinaciones de los 6 equipos de cada grupo
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
        
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [80]:
# mostrar tabla actualizada
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


## 3.2 Octavos

df_fixture_knockout representa los octavos de final, donde "Winners Grouo" es el ganador del grupo y "Runner-up Group" es el 2do puesto del grupo

In [81]:
# octavos: df_fixture_knockout
df_fixture_knockout

,home,away,score,year
48,Winners Group A,Runners-up Group B,Match 49,2022
49,Winners Group C,Runners-up Group D,Match 50,2022
50,Winners Group D,Runners-up Group C,Match 52,2022
51,Winners Group B,Runners-up Group A,Match 51,2022
52,Winners Group E,Runners-up Group F,Match 53,2022
53,Winners Group G,Runners-up Group H,Match 54,2022
54,Winners Group F,Runners-up Group E,Match 55,2022
55,Winners Group H,Runners-up Group G,Match 56,2022


1 Actualizamos el fixture de octavos con el 1er puesto (group winner) y el 2do puesto (runners up)

In [82]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    
    df_fixture_knockout.replace({f'Winners {group}': group_winner,
                                 f'Runners-up {group}': runners_up}, inplace=True)
    
    df_fixture_knockout['winner'] = '?'

In [83]:
df_fixture_knockout

,home,away,score,year,winner
48,Netherlands,Wales,Match 49,2022,?
49,Argentina,Denmark,Match 50,2022,?
50,France,Poland,Match 52,2022,?
51,England,Senegal,Match 51,2022,?
52,Germany,Belgium,Match 53,2022,?
53,Brazil,Portugal,Match 54,2022,?
54,Croatia,Spain,Match 55,2022,?
55,Uruguay,Switzerland,Match 56,2022,?


2 Creamos la función get_winners y actualizamos el df de octavos con los ganadores de cada partido

In [84]:
# crear funcion get_winners
def get_winners(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        
        if points_home > points_away:
            winner = home
        else:
            winner = away
            
        df_fixture_updated.loc[index, 'winner'] = winner
    
    return df_fixture_updated

In [85]:
get_winners(df_fixture_knockout)

,home,away,score,year,winner
48,Netherlands,Wales,Match 49,2022,Netherlands
49,Argentina,Denmark,Match 50,2022,Argentina
50,France,Poland,Match 52,2022,France
51,England,Senegal,Match 51,2022,England
52,Germany,Belgium,Match 53,2022,Germany
53,Brazil,Portugal,Match 54,2022,Brazil
54,Croatia,Spain,Match 55,2022,Spain
55,Uruguay,Switzerland,Match 56,2022,Uruguay


## 3.3 Cuartos de Final

In [86]:
def update_table(df_fixture_round1, df_fixture_round2):
    for index, row in df_fixture_round1.iterrows():
        winner = df_fixture_round1.loc[index, 'winner']
        match = df_fixture_round1.loc[index, 'score']
        df_fixture_round2.replace({f'Winners {match}': winner}, inplace=True)
    
    df_fixture_round2['winner'] = '?'
    return df_fixture_round2

In [87]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,away,score,year,winner
56,Germany,Brazil,Match 58,2022,?
57,Netherlands,Argentina,Match 57,2022,?
58,Spain,Uruguay,Match 60,2022,?
59,England,France,Match 59,2022,?


In [88]:
get_winners(df_fixture_quarter)

,home,away,score,year,winner
56,Germany,Brazil,Match 58,2022,Brazil
57,Netherlands,Argentina,Match 57,2022,Netherlands
58,Spain,Uruguay,Match 60,2022,Spain
59,England,France,Match 59,2022,France


## 3.4 Semifinal

In [89]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,away,score,year,winner
60,Netherlands,Brazil,Match 61,2022,?
61,France,Spain,Match 62,2022,?


In [90]:
get_winners(df_fixture_semi)

,home,away,score,year,winner
60,Netherlands,Brazil,Match 61,2022,Brazil
61,France,Spain,Match 62,2022,France


## 3.5 Final

In [91]:
update_table(df_fixture_semi, df_fixture_final)

,home,away,score,year,winner
62,Losers Match 61,Losers Match 62,Match 63,2022,?
63,Brazil,France,Match 64,2022,?


In [92]:
get_winners(df_fixture_final)

,home,away,score,year,winner
62,Losers Match 61,Losers Match 62,Match 63,2022,Losers Match 62
63,Brazil,France,Match 64,2022,Brazil
